## Preprocessing step

In [ ]:
import pandas as pd
import numpy as np
import random
import ast
import json
import os
import sys
import spotipy
import spotipy.util as util
import requests
from spotipy import oauth2

In [ ]:
def infos(content = False):   
    
    if content == False:
        global client_id
        global client_secret
        global redirect_uri
        global username
        global token
        global user_id
        global scope
        
        client_id='client_id'
        user_id='user_id' 
        client_secret='client_secret'
        redirect_uri='redirect_uri'
        username = 'username'
        scope = 'playlist-modify-public user-read-email'

        token = util.prompt_for_user_token(username,
                                            scope,
                                            client_id=client_id,
                                            client_secret=client_secret,
                                            redirect_uri=redirect_uri)
                                       
    print('Informações: client_id, user_id, client_secret, redirect_uri, username, scope, token')
    
# infos()


### Genre filter

In [ ]:
data_GI = pd.read_csv('spotify_tabular_data.csv',index_col=0)

data_GI = data_[['track', 'track_id', 'artist', 'artist_id', 'popularity', 'album',
       'album_id', 'track_year', 'duration_ms', 'uri', 'preview_url', 'energy',
       'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness',
       'time_signature', 'danceability', 'key', 'loudness', 'valence', 'mode']]

In [ ]:
def genre_filter(data_info):
    infos()
    chaves_ok = ['Falamansa','Rastapé',
    'Circuladô de Fulô','Estakazero',
    'Trio Virgulino','Trio Nordestino',
    'Forróçacana','Bicho de Pé','Trio Forrozão',
    'Forrueiros','Raiz do Sana','Chama Chuva',
                 
    'Luiz Gonzaga', 'Rastapé','Trio Dona Zefa'
    ,'Dominguinhos','Santanna "O cantador"',
    'Trio Pé De Serra', 'Severino Januário', 
    'Pe de Serra Forro Band']
    
    
    ID_artista = list(data_info.artist_id)

    sp = spotipy.Spotify(auth=token)
    genres_ok = ['FORRÓ UNIVERSITÁRIO','FORRÓ TRADICIONAL', 'FORRÓ PÉ DE SERRA',
                 'FORRO UNIVERSITARIO','FORRO TRADICIONAL', 'FORRO PE DE SERRA']
    forro = []

    artista_exclude = []
    genres_exclude = []



    for artista in ID_artista:

        artista_info=sp.artist(artista)

        if artista_info['name'] in chaves_ok:
            forro.append(1.1)
        else:
            artista_info = sp.artist(artista)
            k=0
            for genre in artista_info['genres']:
                if genre.upper() in genres_ok:
                    k+=1
            if k==0:
                forro.append(0)
                artista_exclude.append(artista_info['name'])
                genres_exclude.append(artista_info['genres'])

                print("All " +artista_info['name']+ 
                      " genres are out of scope"+ ". Genres: ", artista_info['genres'])
                print()
            else:
                forro.append(k/len(artista_info['genres']))
                print(artista_info['name'] + ' ok!!!' + ". Genres: ", artista_info['genres'] )

    print('Total artists: ',len( list(set(ID_artista))))
    print('Excluded artists: ',len( list(set(artista_exclude))))

    print('Total songs: ',len(ID_artista))
    print('Excluded songs: ',len(artista_exclude))
    
    
    data_info['genre_filt'] = forro

    data_filt = data_info[data_info.genre_filt > 0.3]
    
    return data_filt

In [ ]:
data_filt_genre = genre_filter(data_GI)

### Data organization

In [ ]:
data_filt = data_filt_genre.copy() 

In [ ]:
data_filt.columns

In [ ]:
min_tempo=81
max_tempo = 171


div = 10

list_tempo = ['[60_81)']

values_tempo =[81]
temp = 81
for i in range(1,10):
    
    if i==9:
        list_tempo.append('[169_213)')
    else:
        list_tempo.append('['+str(temp)+'_' +str(temp+div+1)+')')
        values_tempo.append(temp+div+1)
    temp = temp+div+1
values_tempo.append(213)   
print(list_tempo,values_tempo)

data_filt['tempo_bins'] = data_filt['tempo']


In [ ]:
lista_BPM =[]
lista_BPM_min = []

for BPM in data_filt.tempo:
    faixa = 0
    i=0
    for value in values_tempo:
        if BPM>=faixa and BPM<value:
            lista_BPM.append(list_tempo[i])
            lista_BPM_min.append(values_tempo[i])
        i+=1
        faixa = value 
data_filt['tempo_bins'] = lista_BPM
data_filt['tempo_bins_max'] =lista_BPM_min

In [ ]:
data_filt.popularity = data_filt.popularity*-1 
data_filt.genre_filt = data_filt.genre_filt*-1 
data_play = data_filt.sort_values(by=['tempo_bins_max','popularity','genre_filt'])
data_play.popularity = data_play.popularity*-1 
data_play.genre_filt = data_play.genre_filt*-1 
data_play=data_play.reset_index(drop=True)

In [ ]:

BPM_count = list(data_play[['tempo_bins_max']].value_counts(sort=False))
folds = []
valor = random.sample(range(0, 20), 20)
j = 0
i = 0
k = 1
for BPM in data_play.tempo_bins_max:
    print(k,BPM_count[j])
    if k==BPM_count[j]:
        print(BPM_count[j])
        j+=1
        k=0
        valor = random.sample(range(0, 20), 20)
        
    folds.append(valor[i])
    
    i+=1
    if i == 20:
        i=0
    k+=1

data_play['folds'] = folds

In [ ]:
folds = []

j = 0
i = 0
k = 1
random.seed(123)
valor = random.sample(range(0, 20), 20)
for BPM in data_play.tempo_bins_max:
    print(k,BPM_count[j])
    if k==BPM_count[j]:
        
        print(BPM_count[j])
        j+=1

        k=0
        
    
    if k<100:
        folds.append(valor[i])
        i+=1
        if i == 20:
            i=0       
    else: 
        folds.append(None)
        print(None)
    
    
    k+=1
data_fold = data_play.copy()
data_fold['folds'] = folds
data_fold.dropna(inplace=True)
data_fold = data_fold.reset_index(drop=True)

### Manual Filter

#### Playlists creation 

In [ ]:
def criar_playlist(n):

    info = [str('Playlist ' +str(n)),'Forroset manual filter']

    
    """Criação de uma nova playlist"""
    request_body = json.dumps({
        "name": info[0],
        "description": info[1],
        "public": True
    })

    query = "https://api.spotify.com/v1/users/{}/playlists".format(
        user_id)
    response = requests.post(
        query,
        data=request_body,
        headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer {}".format(token)
        }
    )
    response_json = response.json()
    
    # playlist id
    return response_json['id']


def add_musicas_id (tracks,playlist_id):
        
    infos()
    sp = spotipy.Spotify(auth=token)
    
    for track in tracks:
        print(track)
        result = sp.user_playlist_add_tracks(username, playlist_id, 
                                                 track)
        print("Can't get token for", username)

In [ ]:
teste = False
if teste == True:
    playlists  = pd.DataFrame()
    ids = []
    listas = []
    for i in range(0,20):
        ids.append(criar_playlist(i))
        listas.append(i)

    playlists["Playlist"]=listas
    playlists["play_id"] = ids
    playlists.to_csv('manual_filt_playlists_ID.csv')

In [ ]:
playlists = pd.read_csv('manual_filt_playlists_ID.csv',index_col=0)
playlists.shape

#### Adding songs to playlists

In [ ]:
no = False
if no ==True    
    for i in range(0,20):    
        print(i)

        id_play = playlists.iloc[i].play_id
        data_part = data_fold[data_fold.playlist.astype(int)==i]
        tracks = list(data_part.id)
        add_musicas(tracks,id_play)

#### Reading playlists after manual evaluation

In [ ]:
def get_id_by_playlist(playlist_id):
    infos()
    offset = 0
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    songs = []
    ids =[]
    
    dat = pd.DataFrame()
    
    for i in range(0,playlists.shape[0]):    
        
        playlist_id = playlists.iloc[i].play_id
        content = sp.user_playlist_tracks(username, playlist_id, fields=None,
                                          limit=100, offset=offset, market='BR')
        
        result = pd.DataFrame(content['items']).copy()
#                 
        dat = pd.concat([dat,result]).copy()
    
        print('dat',dat.shape)

        
            

    dat = dat.reset_index(drop=True)    
    return dat

In [ ]:
print('Are you sure you want to read data from playlists that were manually rated??')
print('Can you guarantee that no more songs were taken down?')
x = input()
if x=='YES':
    playlists = pd.read_csv('manual_filt_playlists_ID.csv',index_col=0)
    manual_eval= get_id_by_playlist(playlists)

id_musica = []
for i in range(0,manual_eval.shape[0]):

    valor = manual_eval.track.values[i]    
    
    id_musica.append(valor['id'])
    
manual_filt = pd.DataFrame()  
manual_filt['track_id'] = id_musica
manual_filt["manual_filt"] = list(np.ones(manual_filt.shape[0]))
print(manual_filt.shape,manual_filt.manual_filt.sum())
manual_filt.head()

#### Merging data before and after the manual filter

In [ ]:
print(data_fold.shape)

data_manual = pd.DataFrame()  
data_manual = data_fold.merge(manual_filt, how='left',on='track_id')
data_manual = data_manual.fillna(0)
print(data_manual.shape,data_manual.manual_filt.sum())
data_manual.head()

#### Merging FO data with GI, AF an AA data

In [ ]:
data_GI_AF_AA_FO = data_play.merge(data_manual, how='left',on='track_id')

In [ ]:
data_GI_AF_AA_FO.to_csv('spotify_preprocessing_data.csv')